In [1]:
import json

examples = []
with open('./alphanli/train.jsonl') as f:
    for line in f:
        if line:
            examples.append(json.loads(line))
print(len(examples))

labels = []
with open('./alphanli/train-labels.lst') as f:
    for line in f:
        if line:
            labels.append(int(line))
print(len(labels))

169654
169654


In [2]:
from collections import defaultdict

dup_hyp_examples = []
stories = defaultdict(dict)
for (example, label) in zip(examples, labels):
    if example['hyp1'] == example['hyp2']:
        dup_hyp_examples.append((label, example))
        continue
    if example['story_id'] not in stories:
        stories[example['story_id']]['cnt'] = 1
        stories[example['story_id']]['obs1'] = example['obs1']
        stories[example['story_id']]['obs2'] = example['obs2']
        stories[example['story_id']]['hypes'] = defaultdict(lambda: [0, 0])
        stories[example['story_id']]['hypes'][example['hyp1']][int(label==1)] += 1
        stories[example['story_id']]['hypes'][example['hyp2']][int(label==2)] += 1
    else:
        stories[example['story_id']]['cnt'] += 1
        assert stories[example['story_id']]['obs1'] == example['obs1']
        assert stories[example['story_id']]['obs2'] == example['obs2']
        stories[example['story_id']]['hypes'][example['hyp1']][int(label==1)] += 1
        stories[example['story_id']]['hypes'][example['hyp2']][int(label==2)] += 1
print(len(dup_hyp_examples))
print(len(stories))

52
17801


In [3]:
from collections import Counter

confused_hypes = []
n_hyp_counter = Counter()
n_pos_hyp_counter = Counter()
for story_id, story in stories.items():
    n_hyp_counter[len(story['hypes'])] += 1
    n_pos_hyp = 0
    for hyp, counters in story['hypes'].items():
        assert sum(counters) > 0
        if counters[0] == 0:
            n_pos_hyp += 1
        if 0 not in counters:
            confused_hypes.append((story_id, hyp, counters))
            print(story_id, '\n', hyp, counters)
    n_pos_hyp_counter[n_pos_hyp] += 1
print(len(confused_hypes))

print('\nn_hyp count CP')
cum_sum = 0
for num in sorted(list(n_hyp_counter.keys())):
    cum_sum += n_hyp_counter[num]
    print(num, n_hyp_counter[num], cum_sum / len(stories) * 100)
print('mean', sum([k * v for k, v in n_hyp_counter.items()]) / sum(n_hyp_counter.values()))

print('\nn_pos_hyp count P')
for num in sorted(list(n_pos_hyp_counter.keys())):
    print(num, n_pos_hyp_counter[num], n_pos_hyp_counter[num] / len(stories) * 100)
print('mean', sum([k * v for k, v in n_pos_hyp_counter.items()]) / sum(n_pos_hyp_counter.values()))

0c006a8d-592e-404a-a9a1-3d6afd8bbc24-1 
 I moved to Guam. [1, 3]
3271c78f-1139-4c15-97bb-30d1e2763a3e-1 
 Hannah was excited to go. [1, 3]
382ae42d-ca55-45eb-952f-7ab95af19a11-1 
 Amelia answered and asked who it was. [1, 3]
427b97a8-6d24-4685-9425-aaf76d0be576-1 
 She decided to dye it black. [1, 1]
78dcc7e0-e50f-4f1d-a7bc-d3d65a22f6b7-1 
 Travis hated his new job. [1, 3]
8e936865-1c65-49c0-b73b-ddf9f2edca76-1 
 Jeremy was early for the interview. [1, 3]
8e936865-1c65-49c0-b73b-ddf9f2edca76-1 
 Jeremy was late for the interview. [1, 3]
90b50da1-b856-4870-b3d1-9139a0365253-1 
 Briana didn't know how to write her name. [1, 3]
8

n_hyp count CP
2 145 0.8145609797202404
3 34 1.005561485309814
4 365 3.056008089433178
5 2 3.067243413291388
6 177 4.061569574742991
7 120 4.735689006235605
8 1606 13.757654064378405
9 37 13.965507555755293
10 3494 33.593618336048536
11 472 36.24515476658615
12 1130 42.593112746474915
13 116 43.24476153025111
14 566 46.4243581821246
15 1905 57.12600415706983
16 

In [5]:
tmp = stories['3271c78f-1139-4c15-97bb-30d1e2763a3e-1']
tmp1 = stories['382ae42d-ca55-45eb-952f-7ab95af19a11-1']
tmp2 = stories['0c006a8d-592e-404a-a9a1-3d6afd8bbc24-1 ']